In [1]:
# Cell 1: Imports and setup
import pandas as pd
import numpy as np
import os
from math import pi

input_dir = "data/processed/filled_missing/"
output_dir = "data/processed/feature_engineered/"
os.makedirs(output_dir, exist_ok=True)


In [2]:
# Cell 2: Helper function to determine season
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Summer"
    elif month in [6, 7, 8, 9]:
        return "Monsoon"
    else:
        return "Post-Monsoon"


In [3]:
# Cell 3: Process all .xlsx files and add date-based + Fourier features
for filename in os.listdir(input_dir):
    if filename.endswith(".xlsx"):
        in_path = os.path.join(input_dir, filename)
        out_path = os.path.join(output_dir, filename)

        excel = pd.ExcelFile(in_path)
        writer = pd.ExcelWriter(out_path, engine='openpyxl')

        for sheet in excel.sheet_names:
            df = excel.parse(sheet)

            # Ensure 'Date' is datetime
            df['Date'] = pd.to_datetime(df['Date'])

            # Extract date-based features
            df['day'] = df['Date'].dt.day
            df['month'] = df['Date'].dt.month
            df['year'] = df['Date'].dt.year
            df['day_of_year'] = df['Date'].dt.dayofyear
            df['season'] = df['month'].apply(get_season)

            # Add Fourier features for seasonality
            df['sin_day'] = np.sin(2 * pi * df['day_of_year'] / 365.25)
            df['cos_day'] = np.cos(2 * pi * df['day_of_year'] / 365.25)

            # Write to new Excel
            df.to_excel(writer, sheet_name=sheet, index=False)

        writer._save()
        print(f"Features added and saved: {filename}")


Features added and saved: Aurangabad_weather.xlsx
Features added and saved: Gangapur_weather.xlsx
Features added and saved: Kannad_weather.xlsx
Features added and saved: Khuldabad_weather.xlsx
Features added and saved: Paithan_weather.xlsx
Features added and saved: Phulambri_weather.xlsx
Features added and saved: Sillod_weather.xlsx
Features added and saved: Soyagaon_weather.xlsx
Features added and saved: Vaijapur_weather.xlsx


In [4]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os

input_dir = "data/processed/feature_engineered/"

# Initialize encoder
le = LabelEncoder()
le.fit(['Winter', 'Summer', 'Monsoon', 'Post-Monsoon'])

for filename in os.listdir(input_dir):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(input_dir, filename)
        
        # Read all sheets at once into a dictionary
        try:
            sheets_dict = pd.read_excel(file_path, sheet_name=None)
        except Exception as e:
            print(f"[ERROR] Could not read {filename}: {e}")
            continue
        
        # Apply label encoding to season column
        for sheet_name, df in sheets_dict.items():
            if 'season' in df.columns:
                df['season_id'] = le.transform(df['season'])
                sheets_dict[sheet_name] = df  # update sheet

        # Save updated sheets back to same file
        try:
            with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
                for sheet_name, df in sheets_dict.items():
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"Encoded 'season' in: {filename}")
        except Exception as e:
            print(f"[ERROR] Could not write {filename}: {e}")


[OK] Encoded 'season' in: Aurangabad_weather.xlsx
[OK] Encoded 'season' in: Gangapur_weather.xlsx
[OK] Encoded 'season' in: Kannad_weather.xlsx
[OK] Encoded 'season' in: Khuldabad_weather.xlsx
[OK] Encoded 'season' in: Paithan_weather.xlsx
[OK] Encoded 'season' in: Phulambri_weather.xlsx
[OK] Encoded 'season' in: Sillod_weather.xlsx
[OK] Encoded 'season' in: Soyagaon_weather.xlsx
[OK] Encoded 'season' in: Vaijapur_weather.xlsx
